In [7]:
import openmc
import openmc.mgxs
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas
import h5py
%matplotlib inline

In [8]:
import os

In [9]:
os.environ['PATH'] += r':/home/jae123p/miniconda3/envs/openmc-env/bin'

%env OPENMC_CROSS_SECTIONS=/home/jae123p/endfb80/endfb-viii.0-hdf5/cross_sections.xml

env: OPENMC_CROSS_SECTIONS=/home/jae123p/endfb80/endfb-viii.0-hdf5/cross_sections.xml


In [10]:
model = openmc.Model()

In [11]:
nickel = openmc.Material(name = "nickel")
nickel.set_density('sum')
nickel.add_element('Ni', 9.1322e-02)

materials = openmc.Materials([nickel])

In [12]:
model.materials = openmc.Materials([nickel])

In [13]:
box = openmc.model.RectangularPrism(width=10, height = 10, boundary_type='reflective')
mat_cell = openmc.Cell(name='nickelCell', fill = nickel, region = -box)

In [14]:
root_universe = openmc.Universe()
root_universe.add_cell(mat_cell)
model.geometry = openmc.Geometry(root_universe)

In [15]:
bounds = [-10, -10, -10, 10, 10, 10]
uniform_dist = openmc.stats.Box(bounds[:3], bounds[3:])
energy_dist = openmc.stats.Uniform(a=0.0e6,b=20.0e6)
source = openmc.IndependentSource(space=uniform_dist, energy=energy_dist)

In [16]:
batches = 500
inactive = 20
particles = 10000

# Instantiate a Settings object
settings = openmc.Settings()

settings.source = source
settings.batches = batches
settings.inactive = inactive
settings.particles = particles
settings.run_mode = 'fixed source'

my_path = nickel.name
settings.output = {'tallies': False, 'path':my_path}

model.settings = settings

In [17]:
lanl30_edges= [
    1.00e-5,  # Thermal cutoff (lower bound)
    1.00e+0, 5.00e+0, 1.00e+1, 5.00e+1, 1.00e+2, 3.00e+2, 5.00e+2, 1.00e+3, 2.00e+3,
    3.00e+3, 5.00e+3, 1.00e+4, 2.00e+4, 3.00e+4, 5.00e+4, 8.00e+4, 1.00e+5, 1.50e+5,
    2.00e+5, 3.00e+5, 4.00e+5, 6.00e+5, 8.00e+5, 1.00e+6, 1.50e+6, 2.00e+6, 3.00e+6,
    5.00e+6, 1.00e+7, 2.00e+7  # Upper bound (fast neutrons)
]

groups = openmc.mgxs.EnergyGroups(group_edges=lanl30_edges)

In [18]:
# Initialize MGXS Library
mgxs_lib = openmc.mgxs.Library(model.geometry)

# Add the chosen group structure
mgxs_lib.energy_groups = groups

# Scatttering Format and Legendre Order
mgxs_lib.scatter_format = "legendre"
mgxs_lib.legendre_order = 3

# Specify multi-group cross-section types to compute
mgxs_lib.mgxs_types = ['total', 'absorption', 'scatter', 'fission', 'nu-fission', 'chi', 'reduced absorption', 'scatter matrix', 'nu-scatter matrix',\
                       'consistent nu-scatter matrix', 'multiplicity matrix']

# set uncertainty goal
mgxs_lib.tally_trigger = openmc.Trigger('std_dev', 1e-4)

# Compute cross sections on a nuclide-by-nuclide basis
mgxs_lib.by_nuclide = False

# Specify a "cell" domain type for the cross section tally filters
mgxs_lib.domain_type = 'material'

# Specify the cell domains over which to compute multi-group cross sections
mgxs_lib.domains = model.materials

# Construct all tallies needed for the multi-group cross section library
mgxs_lib.build_library()

tallies = openmc.Tallies()
mgxs_lib.add_to_tallies_file(tallies, merge=True)
model.tallies = tallies

/home/jae123p/miniconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mgxs/library.py:401: RuntimeWarning: The P0 correction will be ignored since the scattering order 3 is greater than zero
  warn(msg, RuntimeWarning)


In [19]:
 model.export_to_model_xml(path = my_path + '/' + mat_cell.name + '.xml')

/home/jae123p/miniconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=1.
  warn(msg, IDWarning)
/home/jae123p/miniconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=2.
  warn(msg, IDWarning)
/home/jae123p/miniconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=12.
  warn(msg, IDWarning)


In [20]:
sp_file = model.run()

[LAPTOP-H8MU6HJU:09004] shmem: mmap: an error occurred while determining whether or not /tmp/ompi.LAPTOP-H8MU6HJU.1000/jf.0/1934753792/shared_mem_cuda_pool.LAPTOP-H8MU6HJU could be created.
[LAPTOP-H8MU6HJU:09004] create_and_attach: unable to create shared memory BTL coordinating structure :: size 134217728 
                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%

In [21]:
# Load the last statepoint file
sp = openmc.StatePoint(sp_file)

In [22]:
# Initialize MGXS Library with OpenMC statepoint data
mgxs_lib.load_from_statepoint(sp)

In [23]:
mgxs_file = mgxs_lib.create_mg_library(xs_type = 'macro')
mgxs_file.export_to_hdf5(filename = 'nickel/nickel.h5')

In [24]:
sp.close()